In [ ]:
TA CAN RUN 6 CELLS FROM HERE TO SEE OUTPUT . ALSO UPDATE THE IMAGE PATH BELOW

In [ ]:
custom_img_path="input.jpg"

# update the custom image path here

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import vgg19
import numpy as np
import cv2
import os
from PIL import Image
from tqdm import tqdm
from glob import glob

In [7]:
class PerceptualLoss(nn.Module):
    def __init__(self):
        super().__init__()
        vgg = vgg19(pretrained=True).features[:16]  # up to relu_4_1
        self.vgg = nn.Sequential(*list(vgg)).eval()
        for param in self.vgg.parameters():
            param.requires_grad = False

    def forward(self, sr, hr):
        return F.l1_loss(self.vgg(sr), self.vgg(hr))

class TotalVariationLoss(nn.Module):
    def forward(self, x):
        return torch.mean(torch.abs(x[:, :, :-1] - x[:, :, 1:])) + \
               torch.mean(torch.abs(x[:, :-1, :] - x[:, 1:, :]))




class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1)
        )

    def forward(self, x):
        return x + 0.1 * self.block(x)

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.fc(x)
        return x * scale

class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_out = torch.mean(x, dim=1, keepdim=True)
        attn = self.conv(torch.cat([max_out, avg_out], dim=1))
        return x * attn


class DenseBlock(nn.Module):
    def __init__(self, channels=64, growth=32):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(5):
            in_ch = channels + i * growth
            self.layers.append(nn.Sequential(
                nn.Conv2d(in_ch, growth, 3, padding=1),
                nn.LeakyReLU(0.2, inplace=True)
            ))
        self.final = nn.Conv2d(channels + 5 * growth, channels, 3, padding=1)

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            features.append(layer(torch.cat(features, dim=1)))
        return x + 0.2 * self.final(torch.cat(features, dim=1))

class RRDBBlock(nn.Module):
    def __init__(self, channels=64):
        super().__init__()
        self.block1 = DenseBlock(channels)
        self.block2 = DenseBlock(channels)
        self.block3 = DenseBlock(channels)

    def forward(self, x):
        return x + 0.2 * self.block3(self.block2(self.block1(x)))


class TransformerBlock(nn.Module):
    def __init__(self, dim, heads=4, mlp_ratio=2.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(dim * mlp_ratio), dim)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        x_flat = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x_norm = self.norm1(x_flat)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x = x_flat + attn_out
        x = x + self.mlp(self.norm2(x))
        return x.reshape(B, H, W, C).permute(0, 3, 1, 2)


class SRModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.res_blocks = nn.Sequential(*[ResidualBlock(64) for _ in range(4)])
        self.se = SEBlock(64)
        self.sa = SpatialAttention()
        self.rrdb = RRDBBlock(64)
        self.transformer = TransformerBlock(64)

        self.upsample = nn.Sequential(
            nn.Conv2d(64, 256, 3, padding=1),
            nn.PixelShuffle(2),
            nn.Conv2d(64, 256, 3, padding=1),
            nn.PixelShuffle(2),
            nn.Conv2d(64, 3, 3, padding=1)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.res_blocks(x)
        x = self.se(x)
        x = self.sa(x)
        x = self.rrdb(x)
        x = self.transformer(x)
        return self.upsample(x)




class SRDatasetFullImage(Dataset):
    def __init__(self, hr_dirs, lr_dir):
        self.hr_paths = []
        for hr_dir in hr_dirs:
            self.hr_paths += sorted(glob(os.path.join(hr_dir, "*.png")))
        
        self.lr_paths = sorted(glob(os.path.join(lr_dir, "*.png")))
        
        self.hr_paths = sorted(self.hr_paths, key=lambda x: os.path.basename(x))
        self.lr_paths = sorted(self.lr_paths, key=lambda x: os.path.basename(x))

        assert len(self.hr_paths) == len(self.lr_paths), "Mismatch: HR=" + str(len(self.hr_paths)) + ", LR=" + str(len(self.lr_paths))


    def __len__(self):
        return len(self.hr_paths)

    def __getitem__(self, idx):
        hr = cv2.imread(self.hr_paths[idx])
        lr = cv2.imread(self.lr_paths[idx])

        hr = cv2.resize(hr, (384, 384))
        lr = cv2.resize(lr, (96, 96))

        hr_tensor = torch.FloatTensor(hr / 127.5 - 1.0).permute(2, 0, 1)
        lr_tensor = torch.FloatTensor(lr / 127.5 - 1.0).permute(2, 0, 1)
        return lr_tensor, hr_tensor

def sr_patchwise_blended(model, lr_image_path, patch_size=96, stride=48, upscale_factor=4, save_name="comparison.png"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    img = cv2.imread(lr_image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # pad image
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    img_padded = np.pad(img, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')
    H, W = img_padded.shape[:2]

    # accumulators
    sr_accum = np.zeros((H * upscale_factor, W * upscale_factor, 3), dtype=np.float32)
    weight_mask = np.zeros_like(sr_accum, dtype=np.float32)

    # sliding window
    for i in range(0, H - patch_size + 1, stride):
        for j in range(0, W - patch_size + 1, stride):
            patch = img_padded[i:i+patch_size, j:j+patch_size]
            patch_tensor = torch.FloatTensor(patch / 127.5 - 1.0).permute(2, 0, 1).unsqueeze(0).to(device)

            with torch.no_grad():
                sr_patch = model(patch_tensor).clamp(-1, 1)[0]
                sr_patch = ((sr_patch.permute(1, 2, 0).cpu().numpy() + 1) * 127.5)

            # blend into output canvas
            y_start = i * upscale_factor
            x_start = j * upscale_factor
            y_end = y_start + sr_patch.shape[0]
            x_end = x_start + sr_patch.shape[1]

            sr_accum[y_start:y_end, x_start:x_end] += sr_patch
            weight_mask[y_start:y_end, x_start:x_end] += 1.0

    # normalize overlapping regions
    sr_output = np.clip(sr_accum / np.maximum(weight_mask, 1e-8), 0, 255).astype(np.uint8)

    # crop padding
    sr_output = sr_output[:h * upscale_factor, :w * upscale_factor]
    lr_upscaled = cv2.resize(img, (sr_output.shape[1], sr_output.shape[0]), interpolation=cv2.INTER_NEAREST)

    # comparison image
    comparison = np.hstack((lr_upscaled, sr_output))
    cv2.imwrite(save_name, cv2.cvtColor(comparison, cv2.COLOR_RGB2BGR))
    print(" " + save_name + " saved with blended patches.")


    return img, sr_output, lr_upscaled


In [8]:

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SRModel().to(device)

model_path = "sr_model_epoch_43.pth"
model.load_state_dict(torch.load(model_path, map_location=device))

model.eval()
print("Loaded Epoch 43 model on", device)

Loaded Epoch 43 model on cpu


In [9]:

custom_img = cv2.imread(custom_img_path)
c_h,c_w=custom_img.shape[:2]
print("Testing on "+str(c_h)+" px x "+str(c_w)+" px")

Testing on 1282 px x 1920 px


In [12]:
custom_img_resized_dir = "custom_img_degraded_dir"
os.makedirs(custom_img_resized_dir, exist_ok=True)
print("SR generation STARTED (wait till success message)")

if c_h > 200 or c_w > 200:
    target_height = 200
    scale = target_height / c_h
    target_width = int(c_w * scale)

    resized_img = cv2.resize(custom_img, (target_width, target_height), interpolation=cv2.INTER_AREA)
    resized_name = os.path.basename(custom_img_path)
    custom_img_resized_path = os.path.join(custom_img_resized_dir, resized_name)
    cv2.imwrite(custom_img_resized_path, resized_img)
    print("Resized image saved at: " + custom_img_resized_path)


    sr_input_path = custom_img_resized_path
else:
    sr_input_path = custom_img_path

_, sr_img, _ = sr_patchwise_blended(model, sr_input_path)
Image.fromarray(sr_img).save("final_sr_output.png")
print("final_sr_output.png saved.")
print("SR Success")

SR generation STARTED (wait till success message)
Resized image saved at: custom_img_degraded_dir/input.jpg
 comparison.png saved with blended patches.
final_sr_output.png saved.
SR Success


In [ ]:
TA DONT NEED TO RUN BELOW CELLS ELSE THE TRAINING MAY START

In [ ]:
BELOW CODE WAS USED DURING TRAINING

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models import vgg19
import numpy as np
import cv2
import os
from PIL import Image
from tqdm import tqdm


In [6]:
import os

# base dir
base_dir = "/kaggle/working/SuperResolution_Project"

# input hr dir as list
div2k_hr_dir = "/kaggle/input/div2k-dataset/DIV2K_train_HR/DIV2K_train_HR"
flickr2k_hr_dir = "/kaggle/input/flickr2k-dataset/Flickr2K_HR"  

# combine both for training
train_hr_dirs = [div2k_hr_dir, flickr2k_hr_dir]

# output folders (where we can write files)
train_lr_dir = base_dir + "/Combined_LR"
progress_dir = base_dir + "/training_progress"
results_dir = base_dir + "/results"
model_dir = base_dir + "/models"


for d in [train_lr_dir, progress_dir, results_dir, model_dir]:
    os.makedirs(d, exist_ok=True)


In [7]:
import cv2
import numpy as np

def degrade_image(img):
    h, w = img.shape[:2]

    # gaussian blur
    k = np.random.choice([5, 7, 9])
    sigma = np.random.uniform(0.8, 2.5)
    img_blur = cv2.GaussianBlur(img, (k, k), sigma)

    # downsample to simulate low-res
    img_small = cv2.resize(img_blur, (w // 4, h // 4), interpolation=cv2.INTER_LINEAR)

    # add Gaussian noise 
    noise_std = np.random.uniform(3, 15)
    noise = np.random.normal(0, noise_std, img_small.shape).astype(np.float32)
    img_noisy = np.clip(img_small.astype(np.float32) + noise, 0, 255).astype(np.uint8)

    # add haze overlay
    haze_value = np.random.randint(180, 230)
    haze = np.full_like(img_noisy, haze_value)
    alpha = np.random.uniform(0.75, 0.95)
    img_hazy = cv2.addWeighted(img_noisy, alpha, haze, 1 - alpha, 0)

   
    return img_hazy


In [8]:
import glob
import os


div2k_hr_dir = "/kaggle/input/div2k-dataset/DIV2K_train_HR/DIV2K_train_HR"
flickr2k_hr_dir = "/kaggle/input/flickr2k/Flickr2K"  

# lr output folder
lr_dir = "/kaggle/working/SuperResolution_Project/Combined_LR"
os.makedirs(lr_dir, exist_ok=True)

# get all hr images
hr_paths = []
for d in [div2k_hr_dir, flickr2k_hr_dir]:
    hr_paths += sorted(glob.glob(f"{d}/*.png"))

print("Total HR images found: " + str(len(hr_paths)))



✅ Total HR images found: 3450


In [ ]:
import os
import cv2
from glob import glob
from tqdm import tqdm

# loop
print("Total HR images found: " + str(len(hr_paths)))


for hr_path in tqdm(hr_paths, desc="Generating LR images"):
    img = cv2.imread(hr_path)
    
    if img is None:
        print("Skipped unreadable image: " + hr_path)

        continue

    # degrade image 
    lr_img = degrade_image(img)

    # save lr image using same filename
    lr_name = os.path.basename(hr_path)
    cv2.imwrite(os.path.join(lr_dir, lr_name), lr_img)


In [24]:
import torch.nn as nn
from torchvision.models import vgg19
import torch.nn.functional as F

class PerceptualLoss(nn.Module):
    def __init__(self):
        super().__init__()
        vgg = vgg19(pretrained=True).features[:16]  # up to relu_4_1
        self.vgg = nn.Sequential(*list(vgg)).eval()
        for param in self.vgg.parameters():
            param.requires_grad = False

    def forward(self, sr, hr):
        return F.l1_loss(self.vgg(sr), self.vgg(hr))

class TotalVariationLoss(nn.Module):
    def forward(self, x):
        return torch.mean(torch.abs(x[:, :, :-1] - x[:, :, 1:])) + \
               torch.mean(torch.abs(x[:, :-1, :] - x[:, 1:, :]))


In [25]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1)
        )

    def forward(self, x):
        return x + 0.1 * self.block(x)

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.fc(x)
        return x * scale

class SpatialAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_out = torch.mean(x, dim=1, keepdim=True)
        attn = self.conv(torch.cat([max_out, avg_out], dim=1))
        return x * attn


In [26]:
class DenseBlock(nn.Module):
    def __init__(self, channels=64, growth=32):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(5):
            in_ch = channels + i * growth
            self.layers.append(nn.Sequential(
                nn.Conv2d(in_ch, growth, 3, padding=1),
                nn.LeakyReLU(0.2, inplace=True)
            ))
        self.final = nn.Conv2d(channels + 5 * growth, channels, 3, padding=1)

    def forward(self, x):
        features = [x]
        for layer in self.layers:
            features.append(layer(torch.cat(features, dim=1)))
        return x + 0.2 * self.final(torch.cat(features, dim=1))

class RRDBBlock(nn.Module):
    def __init__(self, channels=64):
        super().__init__()
        self.block1 = DenseBlock(channels)
        self.block2 = DenseBlock(channels)
        self.block3 = DenseBlock(channels)

    def forward(self, x):
        return x + 0.2 * self.block3(self.block2(self.block1(x)))


In [27]:
class TransformerBlock(nn.Module):
    def __init__(self, dim, heads=4, mlp_ratio=2.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(dim * mlp_ratio), dim)
        )

    def forward(self, x):
        B, C, H, W = x.shape
        x_flat = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x_norm = self.norm1(x_flat)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x = x_flat + attn_out
        x = x + self.mlp(self.norm2(x))
        return x.reshape(B, H, W, C).permute(0, 3, 1, 2)


In [28]:
class SRModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.res_blocks = nn.Sequential(*[ResidualBlock(64) for _ in range(4)])
        self.se = SEBlock(64)
        self.sa = SpatialAttention()
        self.rrdb = RRDBBlock(64)
        self.transformer = TransformerBlock(64)

        self.upsample = nn.Sequential(
            nn.Conv2d(64, 256, 3, padding=1),
            nn.PixelShuffle(2),
            nn.Conv2d(64, 256, 3, padding=1),
            nn.PixelShuffle(2),
            nn.Conv2d(64, 3, 3, padding=1)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.res_blocks(x)
        x = self.se(x)
        x = self.sa(x)
        x = self.rrdb(x)
        x = self.transformer(x)
        return self.upsample(x)


In [29]:
import os
import cv2
import torch
from glob import glob
from torch.utils.data import Dataset

class SRDatasetFullImage(Dataset):
    def __init__(self, hr_dirs, lr_dir):
        self.hr_paths = []
        for hr_dir in hr_dirs:
            self.hr_paths += sorted(glob(os.path.join(hr_dir, "*.png")))
        
        self.lr_paths = sorted(glob(os.path.join(lr_dir, "*.png")))
        
        # ensure hr and lr match by filename
        self.hr_paths = sorted(self.hr_paths, key=lambda x: os.path.basename(x))
        self.lr_paths = sorted(self.lr_paths, key=lambda x: os.path.basename(x))

        assert len(self.hr_paths) == len(self.lr_paths), "❌ Mismatch: HR=" + str(len(self.hr_paths)) + ", LR=" + str(len(self.lr_paths))

    def __len__(self):
        return len(self.hr_paths)

    def __getitem__(self, idx):
        hr = cv2.imread(self.hr_paths[idx])
        lr = cv2.imread(self.lr_paths[idx])

        # resize
        hr = cv2.resize(hr, (384, 384))
        lr = cv2.resize(lr, (96, 96))

        
        hr_tensor = torch.FloatTensor(hr / 127.5 - 1.0).permute(2, 0, 1)
        lr_tensor = torch.FloatTensor(lr / 127.5 - 1.0).permute(2, 0, 1)
        return lr_tensor, hr_tensor


In [30]:
from torch.utils.data import DataLoader

hr_dirs = [
    "/kaggle/input/div2k-dataset/DIV2K_train_HR/DIV2K_train_HR",
    "/kaggle/input/flickr2k/Flickr2K"
]

lr_dir = "/kaggle/working/SuperResolution_Project/Combined_LR"

train_loader = DataLoader(
    SRDatasetFullImage(hr_dirs, lr_dir),
    batch_size=1,  
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
print("Om")

Om


In [ ]:
import os

log_dir = "/kaggle/working/training_progress"
os.makedirs(log_dir, exist_ok=True)  

with open(os.path.join(log_dir, "loss_log.txt"), "a") as f:
    f.write("Logs\n")


In [ ]:
model = SRModel().to(device)
model_path = "/kaggle/working/models/sr_model_epoch_33.pth"
model.load_state_dict(torch.load(model_path))
model.eval()
print(" Loaded Epoch 33 model")


In [ ]:
import cv2
import torch
import torch.nn as nn

def lab_color_loss(sr, hr):
    # convert [-1, 1] tensors to [0, 255] nump
    sr_img = ((sr.detach().cpu().permute(1, 2, 0).numpy() + 1) * 127.5).astype(np.uint8)
    hr_img = ((hr.detach().cpu().permute(1, 2, 0).numpy() + 1) * 127.5).astype(np.uint8)

    sr_lab = cv2.cvtColor(sr_img, cv2.COLOR_RGB2LAB)
    hr_lab = cv2.cvtColor(hr_img, cv2.COLOR_RGB2LAB)

    # convert to tensor again for loss
    sr_tensor = torch.FloatTensor(sr_lab / 255.).permute(2, 0, 1).to(sr.device)
    hr_tensor = torch.FloatTensor(hr_lab / 255.).permute(2, 0, 1).to(hr.device)

    return nn.L1Loss()(sr_tensor, hr_tensor)


In [ ]:
import torch.nn.functional as F

def laplacian_loss(sr, hr):
    lap_filter = torch.tensor([[0, -1, 0],
                               [-1, 4, -1],
                               [0, -1, 0]], dtype=torch.float32, device=sr.device)
    lap_filter = lap_filter.view(1, 1, 3, 3).repeat(3, 1, 1, 1)  # shape [3,1,3,3]
    
    sr_edges = F.conv2d(sr, lap_filter, padding=1, groups=3)
    hr_edges = F.conv2d(hr, lap_filter, padding=1, groups=3)
    
    return F.l1_loss(sr_edges, hr_edges)


In [ ]:
import torch
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim


generator = SRModel().to(device)

optimizer = torch.optim.Adam(generator.parameters(), lr=1e-4)
l1_loss = nn.L1Loss()
vgg_loss = PerceptualLoss().to(device)
tv_loss = TotalVariationLoss().to(device)


for epoch in range(34, 101):  
    generator.train()
    total_loss = 0

    for lr, hr in tqdm(train_loader, desc=f"Epoch {epoch}"):
        lr, hr = lr.to(device), hr.to(device)

        sr = generator(lr)

        # loss = (
        #     0.2 * l1_loss(sr, hr) +
        #     0.5 * vgg_loss(sr, hr) +
        #     0.1 * tv_loss(sr)
        # )
        loss = (
            1.0 * l1_loss(sr, hr) +
            0.5 * vgg_loss(sr, hr) +
            0.1 * tv_loss(sr) +
            0.2 * lab_color_loss(sr[0], hr[0]) +     
            0.05 * laplacian_loss(sr, hr)            
        )


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print("Epoch " + str(epoch) + " | Loss: " + "{:.4f}".format(total_loss / len(train_loader)))

    
    model_path = f"/kaggle/working/models/sr_model_epoch_{epoch}.pth"
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    torch.save(generator.state_dict(), model_path)

    # save sample sr output for visual inspection
    generator.eval()
    with torch.no_grad():
        lr, hr = train_loader.dataset[0]  
        sr = generator(lr.unsqueeze(0).to(device)).clamp(-1, 1)[0]

        sr_img = ((sr.permute(1, 2, 0).cpu().numpy() + 1) * 127.5).astype(np.uint8)
        hr_img = ((hr.permute(1, 2, 0).cpu().numpy() + 1) * 127.5).astype(np.uint8)
        lr_img = ((lr.permute(1, 2, 0).cpu().numpy() + 1) * 127.5).astype(np.uint8)

        
        lr_up = cv2.resize(lr_img, (sr_img.shape[1], sr_img.shape[0]), interpolation=cv2.INTER_NEAREST)
        comparison = np.hstack([lr_up, sr_img, hr_img])
        
        psnr_val = psnr(hr_img, sr_img, data_range=255)
        ssim_val = ssim(hr_img, sr_img, channel_axis=-1, data_range=255)

        
        
        print("Epoch " + str(epoch) + " PSNR: " + "{:.2f}".format(psnr_val) + " | SSIM: " + "{:.4f}".format(ssim_val))

       
        with open(f"/kaggle/working/training_progress/loss_log.txt", "a") as f:
            f.write("Epoch " + str(epoch) + " | Loss: " + "{:.4f}".format(total_loss / len(train_loader)) + 
        " | PSNR: " + "{:.2f}".format(psnr_val) + " | SSIM: " + "{:.4f}".format(ssim_val) + "\n")

        cv2.imwrite("/kaggle/working/epoch_" + str(epoch) + "_sample.jpg", comparison)



below code is fo testing

In [31]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = SRModel().to(device)

model_path = "/kaggle/working/models/sr_model_epoch_43.pth"
model.load_state_dict(torch.load(model_path, map_location=device))

model.eval()
print("Loaded Epoch 43 model on", device)


✅ Loaded Epoch 43 model on cuda


/tmp/ipykernel_31/3788568203.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [32]:
import glob
import os
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

set14_dir = "/kaggle/input/set-5-14-super-resolution-dataset/Set14/Set14"
# set14_dir = "/kaggle/input/heetdave2"
output_dir = "/kaggle/working/results_epoch43_set14"
os.makedirs(output_dir, exist_ok=True)

image_paths = sorted(glob.glob(os.path.join(set14_dir, "*.png")) + glob.glob(os.path.join(set14_dir, "*.jpg")))


In [ ]:
def super_resolve_image(img_path, model, upscale=4):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    h, w = img_rgb.shape[:2]
    lr = cv2.resize(img_rgb, (w // upscale, h // upscale), interpolation=cv2.INTER_CUBIC)

    # normalize [-1, 1]
    lr_tensor = torch.tensor(lr / 127.5 - 1.0, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        sr_tensor = model(lr_tensor).clamp(-1, 1)[0]

    sr_img = ((sr_tensor.permute(1, 2, 0).cpu().numpy() + 1) * 127.5).astype(np.uint8)
    return lr, sr_img, img_rgb


In [ ]:
print("Om")

In [ ]:
import shutil
# shutil.rmtree("/kaggle/working/results_epoch9_set14")

In [ ]:
# import torch
# import cv2
# import numpy as np
# from PIL import Image

# def sr_patchwise(model, lr_image_path, patch_size=96, upscale_factor=4, min_display_width=600):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device).eval()

#     # load and prepare lr image
#     img = cv2.imread(lr_image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     h, w = img.shape[:2]

#     # pad if needed
#     pad_h = (patch_size - h % patch_size) % patch_size
#     pad_w = (patch_size - w % patch_size) % patch_size
#     img_padded = np.pad(img, ((0, pad_h), (0, pad_w), (0, 0)), mode='constant', constant_values=0)
#     H, W = img_padded.shape[:2]

#     # prepare empty sr output image
#     sr_output = np.zeros((H * upscale_factor, W * upscale_factor, 3), dtype=np.uint8)

#     # process each patch
#     for i in range(0, H, patch_size):
#         for j in range(0, W, patch_size):
#             patch = img_padded[i:i+patch_size, j:j+patch_size]
#             patch_tensor = torch.FloatTensor(patch / 127.5 - 1.0).permute(2, 0, 1).unsqueeze(0).to(device)

#             with torch.no_grad():
#                 sr_patch = model(patch_tensor).clamp(-1, 1)[0]
#                 sr_patch = ((sr_patch.permute(1, 2, 0).cpu().numpy() + 1) * 127.5).astype(np.uint8)

#             sr_output[i*upscale_factor:(i+patch_size)*upscale_factor,
#                       j*upscale_factor:(j+patch_size)*upscale_factor] = sr_patch

#     # crop out padding
#     final_sr = sr_output[:h*upscale_factor, :w*upscale_factor]
    

#     # create comparision
#     lr_upscaled = cv2.resize(img, (final_sr.shape[1], final_sr.shape[0]), interpolation=cv2.INTER_NEAREST)
#     comparison = np.hstack((lr_upscaled, final_sr))

#     # upscale whole comparison if it's visually too small
#     if comparison.shape[1] < min_display_width:
#         scale_factor = int(np.ceil(min_display_width / comparison.shape[1]))
#         comparison = cv2.resize(comparison, (comparison.shape[1]*scale_factor, comparison.shape[0]*scale_factor),
#                                 interpolation=cv2.INTER_NEAREST)

#     # save
#     cv2.imwrite("comparison.png", cv2.cvtColor(comparison, cv2.COLOR_RGB2BGR))
#     print("comparison.png saved.")

#     return img, final_sr, lr_upscaled





In [14]:
print("Test")

Test


In [33]:
def sr_patchwise_blended(model, lr_image_path, patch_size=96, stride=48, upscale_factor=4, save_name="comparison.png"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    # load lr image
    img = cv2.imread(lr_image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # pad image to fit patches
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    img_padded = np.pad(img, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')
    H, W = img_padded.shape[:2]

    # prepare accumulators
    sr_accum = np.zeros((H * upscale_factor, W * upscale_factor, 3), dtype=np.float32)
    weight_mask = np.zeros_like(sr_accum, dtype=np.float32)

    # sliding window
    for i in range(0, H - patch_size + 1, stride):
        for j in range(0, W - patch_size + 1, stride):
            patch = img_padded[i:i+patch_size, j:j+patch_size]
            patch_tensor = torch.FloatTensor(patch / 127.5 - 1.0).permute(2, 0, 1).unsqueeze(0).to(device)

            with torch.no_grad():
                sr_patch = model(patch_tensor).clamp(-1, 1)[0]
                sr_patch = ((sr_patch.permute(1, 2, 0).cpu().numpy() + 1) * 127.5)

            # blend into output canvas
            y_start = i * upscale_factor
            x_start = j * upscale_factor
            y_end = y_start + sr_patch.shape[0]
            x_end = x_start + sr_patch.shape[1]

            sr_accum[y_start:y_end, x_start:x_end] += sr_patch
            weight_mask[y_start:y_end, x_start:x_end] += 1.0

    # normalize overlapping regions
    sr_output = np.clip(sr_accum / np.maximum(weight_mask, 1e-8), 0, 255).astype(np.uint8)

    # crop padding
    sr_output = sr_output[:h * upscale_factor, :w * upscale_factor]
    lr_upscaled = cv2.resize(img, (sr_output.shape[1], sr_output.shape[0]), interpolation=cv2.INTER_NEAREST)

    # comparison image
    comparison = np.hstack((lr_upscaled, sr_output))
    cv2.imwrite(save_name, cv2.cvtColor(comparison, cv2.COLOR_RGB2BGR))
    print(" " + save_name + " saved with blended patches.")


    return img, sr_output, lr_upscaled


In [35]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import os
import cv2
import numpy as np

for img_path in image_paths:
    name = os.path.basename(img_path).split('.')[0]

    # load hr image
    hr_img = cv2.imread(img_path)
    hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)

    # downsample
    h, w = hr_img.shape[:2]
    #lr_img = cv2.resize(hr_img, (w // 4, h // 4), interpolation=cv2.INTER_CUBIC)
    lr_img=degrade_image(hr_img)
    # save temp lr
    temp_lr_path = f"/kaggle/working/temp_lr/{name}_lr.png"
    os.makedirs(os.path.dirname(temp_lr_path), exist_ok=True)
    cv2.imwrite(temp_lr_path, cv2.cvtColor(lr_img, cv2.COLOR_RGB2BGR))

    # run
    lr, sr, _ = sr_patchwise_blended(model, temp_lr_path)
    # lr, sr, _ = super_resolve_image(temp_lr_path,model)

    # resize
    hr_resized = cv2.resize(hr_img, sr.shape[:2][::-1])
    min_h = min(lr.shape[0] * 4, sr.shape[0], hr_resized.shape[0])

    # stacking
    sr_resized = cv2.resize(sr, (int(sr.shape[1] * min_h / sr.shape[0]), min_h))
    lr_up = cv2.resize(lr, (sr_resized.shape[1], min_h), interpolation=cv2.INTER_NEAREST)
    hr_resized = cv2.resize(hr_resized, (sr_resized.shape[1], min_h))

    # save
    comp = np.hstack([lr_up, sr_resized, hr_resized])
    save_path = os.path.join(output_dir, f"{name}_compare.jpg")
    cv2.imwrite(save_path, cv2.cvtColor(comp, cv2.COLOR_RGB2BGR))
    print("Saved " + save_path)


    # === PSNR / SSIM ===
    hr_crop = cv2.resize(hr_img, sr.shape[:2][::-1])
    psnr = peak_signal_noise_ratio(hr_crop, sr, data_range=255)
    ssim = structural_similarity(hr_crop, sr, channel_axis=-1, data_range=255)
    print(" " + name + " | PSNR: " + "{:.2f}".format(psnr) + " | SSIM: " + "{:.4f}".format(ssim))



✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/baboon_compare.jpg
📊 baboon | PSNR: 19.51 | SSIM: 0.3490
✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/barbara_compare.jpg
📊 barbara | PSNR: 22.35 | SSIM: 0.5534
✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/bridge_compare.jpg
📊 bridge | PSNR: 22.13 | SSIM: 0.4798
✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/coastguard_compare.jpg
📊 coastguard | PSNR: 22.59 | SSIM: 0.4326
✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/comic_compare.jpg
📊 comic | PSNR: 20.78 | SSIM: 0.5986
✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/face_compare.jpg
📊 face | PSNR: 25.93 | SSIM: 0.5320
✅ comparison.png saved with blended patches.
✅ Saved /kaggle/working/results_epoch43_set14/flowers_compare.j

In [ ]:
!zip -r /kaggle/working/results_epoch43_set14.zip /kaggle/working/results_epoch43_set14


In [ ]:
custom_img_path="/kaggle/input/customimages/trendy-colour-swatches-collage.jpg"
custom_img = cv2.imread(custom_img_path)
c_h,c_w=custom_img.shape[:2]
print("Testing on "+str(c_h)+" px x "+str(c_w)+" px")

In [ ]:
custom_img_resized_dir = "/kaggle/working/custom_img_degraded_dir"
os.makedirs(custom_img_resized_dir, exist_ok=True)
if c_h > 200 or c_w > 200:
    target_height = 200
    scale = target_height / c_h
    target_width = int(c_w * scale)

    resized_img = cv2.resize(custom_img, (target_width, target_height), interpolation=cv2.INTER_AREA)
    resized_name = os.path.basename(custom_img_path)
    custom_img_resized_path = os.path.join(custom_img_resized_dir, resized_name)
    cv2.imwrite(custom_img_resized_path, resized_img)
    print("Resized image saved at: " + custom_img_resized_path)


    sr_input_path = custom_img_resized_path
else:
    sr_input_path = custom_img_path

_, sr_img, _ = sr_patchwise_blended(model, sr_input_path)
Image.fromarray(sr_img).save("final_sr_output.png")
print("final_sr_output.png saved.")


In [ ]:
import torch
from PIL import Image

# cpu
device = torch.device("cpu")

# load
model = SRModel().to(device)
model.load_state_dict(torch.load("/kaggle/working/models/sr_model_epoch_43.pth", map_location=device))
model.eval()


#sr_img = sr_patchwise_blended(model, custom_img_degraded_path)

if c_h<=200 or c_w<=200:
    _, sr_img, _ = sr_patchwise_blended(model, custom_img_path)
    Image.fromarray(sr_img).save("final_sr_output.png")
    print("Saved final_sr_output.png")
else:
    _, sr_img, _ = sr_patchwise_blended(model, custom_img_degraded_path)
    Image.fromarray(sr_img).save("final_sr_output.png")
    print("Saved final_sr_output.png")
    custom_hr_resized = cv2.resize(custom_img, (sr_img.shape[1], sr_img.shape[0]))
    
    psnr = peak_signal_noise_ratio(custom_hr_resized, sr_img, data_range=255)
    ssim = structural_similarity(custom_hr_resized, sr_img, channel_axis=2, data_range=255)
    
    print("PSNR: "+str(psnr)+"dB")
    print("SSIM: "+str(ssim))





In [16]:
GIF_PATH = "/kaggle/input/customimages/zebra5.gif"


In [17]:
import os

os.makedirs("gif_extracted_frames", exist_ok=True)
os.makedirs("gif_sr_frames", exist_ok=True)
os.makedirs("gif_sr_output", exist_ok=True)


In [24]:
from PIL import Image


gif = Image.open(GIF_PATH)
durations = []

for i in range(gif.n_frames):
    gif.seek(i)
    duration = gif.info.get("duration", 80)  
    durations.append(duration)

print("Collected frame durations:", durations[:5], "...")


✅ Collected frame durations: [50, 50, 50, 50, 50] ...


In [25]:
for path in extracted_paths:
    _, sr_img, _ = sr_patchwise_blended(model, path)
    out_path = path.replace("gif_extracted_frames", "gif_sr_frames")
    Image.fromarray(sr_img).save(out_path)

print("Super-resolution applied to all frames")


✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ comparison.png saved with blended patches.
✅ Super-resolution applied to all frames


In [26]:
from PIL import Image

sr_frame_paths = sorted([f"gif_sr_frames/{f}" for f in os.listdir("gif_sr_frames") if f.endswith(".png")])
sr_images = [Image.open(f).convert("RGB") for f in sr_frame_paths]

durations = durations[:len(sr_images)]

output_gif_path = "gif_sr_output/final_sr_output.gif"
sr_images[0].save(
    output_gif_path,
    save_all=True,
    append_images=sr_images[1:],
    duration=durations,  # Use original frame durations
    loop=0
)

print("✅ SR GIF saved with original speed: " + output_gif_path)



✅ SR GIF saved with original speed: gif_sr_output/final_sr_output.gif
